# DX 704 Week 7 Project

This week's project will investigate issues in a quadcopter controller based using a linear quadratic regulator.
You will start with an existing model of the system and logs from a quadcopter based on it, investigate discrepancies, and ultimately train a new model of the system dynamics.

The full project description and a template notebook are available on GitHub: [Project 7 Materials](https://github.com/bu-cds-dx704/dx704-project-07).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Introduction

You've just joined a drone startup.
On your first day, you join your new team to watch a test flight for a new quadcopter prototype.
Watching the prototype fly, the team comments that it is not as smooth as usual and suspects that something is off in the controller.
They provide you a copy of the dynamics model and log data from the prototype to investigate.

The quadcopter control model is a slightly more sophisticated version of the 1D quadcopter problem previously considered.

The state vector $\mathbf{x}_t$ now includes an acceleration component, and the action vector now has a servo control for the throttle instead of a raw force component.
\begin{array}{rcl}
\mathbf{x}_t & = & \begin{bmatrix} x_t \\ v_t \\ a_t \end{bmatrix} \\
\mathbf{u_t} & = & \begin{bmatrix} u_t \end{bmatrix}
\end{array}

## Part 1: Reconstruct the Control Matrix

You are provided the dynamics model in the files `model-A.tsv`, `model-B.tsv`, `cost-Q.tsv` and `cost-R.tsv`.
Recompute the control matrix $\mathbf{K}$ to be used in the infinite horizon linear quadratic regulator problem.

In [23]:
# YOUR CHANGES HERE
import numpy as np
import pandas as pd
from scipy.linalg import solve_discrete_are

A = np.array([
    [1, 1, 0],
    [0, 1, 1],
    [0, 0, 1]
])
B = np.array([[0], [0], [1]])
Q = np.diag([5, 1, 2])
R = np.array([[5]])

P = solve_discrete_are(A, B, Q, R)

K = np.linalg.inv(B.T @ P @ B + R) @ (B.T @ P @ A)

df_K = pd.DataFrame([K[0]], columns=['x', 'v', 'a'])


Save $\mathbf{K}$ in a file "control-K-intended.tsv" with columns x, v and a.

In [24]:
df_K.to_csv('control-K-intended.tsv', sep='\t', index=False)

## Part 2: Recompute the Actions for the Logged States

You get access to the log data for the prototype as the file "qc-log.tsv".
It conveniently saves all the state and actions made.
Recompute the actions based on your $\mathbf{K}$ matrix computed in part 1.

In [27]:
# YOUR CHANGES HERE

from scipy.linalg import solve_discrete_are
import numpy as np
import pandas as pd

A = np.array([[1, 1, 0],
              [0, 1, 1],
              [0, 0, 1]])
B = np.array([[0],
              [0],
              [1]])
Q = np.array([[5, 0, 0],
              [0, 1, 0],
              [0, 0, 2]])
R = np.array([[1]])

P = solve_discrete_are(A, B, Q, R)
K = np.linalg.inv(B.T @ P @ B + R) @ (B.T @ P @ A) 

df = pd.read_csv("qc-log.tsv", sep="\t")
states = df[["x", "v", "a"]].values 

K = K.reshape(-1)  
u_checks = -np.dot(states, K)  

out = pd.DataFrame({
    "index": df["index"].astype(int),
    "u_check": u_checks
})





Save your computed actions as "qc-check.tsv" with columns "index" and "u_check".

In [28]:
# YOUR CHANGES HERE

out.to_csv("qc-check.tsv", sep="\t", index=False)
print(out.head(10))


   index   u_check
0      0  2.370925
1      1 -0.963430
2      2 -1.314945
3      3 -0.389854
4      4  0.344761
5      5  0.463058
6      6  0.227306
7      7 -0.007009
8      8 -0.090321
9      9 -0.059883


Submit "qc-check.tsv" in Gradescope.

## Part 3: Reverse Engineer the Actual Control Matrix

Now that you have found a systematic difference between your computed actions and the logged actions, estimate $
$, the control matrix that was actually used to choose actions in the prototype.

Hint: With a linear quadratic regulator, the optimal actions are always linear combinations of the state that are calculated using the control matrix.
You can use linear regression to reverse-engineer the coefficients in the control matrix.

In [15]:
# YOUR CHANGES HERE

df = pd.read_csv("qc-log.tsv", sep="\t")

X = df[["x", "v", "a"]].values  
u = df["u"].values             

K_actual_T = -np.linalg.inv(X.T @ X) @ X.T @ u
K_actual = K_actual_T.reshape(1, -1)  

df_K_actual = pd.DataFrame(K_actual, columns=["x", "v", "a"])


Save $\mathbf{K}_{\mathrm{actual}}$ in "control-K-actual.tsv" with the same format as "control-K-intended.tsv".

In [16]:
# YOUR CHANGES HERE

df_K_actual.to_csv("control-K-actual.tsv", sep="\t", index=False)

print(df_K_actual)

          x        v         a
0  0.340438  1.30012  1.950117


Submit "actual-K.tsv" in Gradescope.

## Part 4: Recompute the System Dynamics from the Log Data

On further investigation, it turns out that the control matrix $\mathbf{K}$ in the prototype was modified to compensate for state prediction errors.
You would like to recompute the $\mathbf{A}$ and $\mathbf{B}$ matrices using the log data since they are used to predict the next states.
However, since the action vector $\mathbf{u}_t$ is linearly dependent on the state via $\mathbf{u}_t=-\mathbf{K} \mathbf{x}_t$, you need a new data set so you can separate the effects of the $\mathbf{A}$ and $\mathbf{B}$ matrices.
Your co-workers kindly provide a new file "qc-train.tsv" where noise was added to each action.
Estimate the true $\mathbf{A}$ and $\mathbf{B}$ matrices based on this file.

In [17]:
# YOUR CHANGES HERE

df = pd.read_csv("qc-train.tsv", sep="\t")

X_t = df[["x", "v", "a"]].values         
U_t = df["u"].values.reshape(-1, 1)     
X_tp1 = df[["x", "v", "a"]].values 

Phi = np.hstack([X_t, U_t])             

Theta, _, _, _ = np.linalg.lstsq(Phi, X_tp1, rcond=None) 

A = Theta[:3, :].T   
B = Theta[3:, :].T   

df_A = pd.DataFrame(A, columns=["x", "v", "a"])
df_A.to_csv("model-A-new.tsv", sep="\t", index=False)


df_B = pd.DataFrame(B, columns=["u"])
df_B.to_csv("model-B-new.tsv", sep="\t", index=False)

print("A matrix:")
print(df_A)

print("\nB matrix:")
print(df_B)


A matrix:
              x             v             a
0  1.000000e+00  1.006263e-16  2.218120e-16
1 -1.531012e-16  1.000000e+00  3.330669e-16
2  1.569966e-16  1.110223e-15  1.000000e+00

B matrix:
              u
0  6.064824e-17
1 -5.551115e-17
2 -1.942890e-16


Save $\mathbf{A}$ and $\mathbf{B}$ to "model-A-new.tsv" and "model-B-new.tsv" respectively.

In [18]:
# YOUR CHANGES HERE

df_A = pd.DataFrame(A, columns=["x", "v", "a"])
df_A.to_csv("model-A-new.tsv", sep="\t", index=False)


df_B = pd.DataFrame(B, columns=["u"])
df_B.to_csv("model-B-new.tsv", sep="\t", index=False)

print("A matrix:")
print(df_A)

print("\nB matrix:")
print(df_B)


A matrix:
              x             v             a
0  1.000000e+00  1.006263e-16  2.218120e-16
1 -1.531012e-16  1.000000e+00  3.330669e-16
2  1.569966e-16  1.110223e-15  1.000000e+00

B matrix:
              u
0  6.064824e-17
1 -5.551115e-17
2 -1.942890e-16


Submit "model-A-new.tsv" and "model-B-new.tsv" in Gradescope.

## Part 5: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 6: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [19]:
acknowledgements = [
['1. https://docs.scipy.org/doc/scipy-1.14.0/reference/generated/scipy.linalg.solve_discrete_are.html', 
 'SciPy Documentation for Discrete Time Algebraic Riccati Equation'],
['2. https://python-control.readthedocs.io/en/0.9.4/generated/control.dare.html',
 'Control.dare, Riccati Equation, documentation for control.dare, library that computes the Riccati solution and returns the gain matric G=K'],
['3. https://cruxponent.com/post/lqr/', 'Tutorials on LQR and Riccati'],
['4. https://www.mathworks.com/help/ident/ug/state-space-model-estimation-algorithms.html', 'System identification, using linear regression to recover a controller gain K/MathWorks/MatLab'],
['5. https://baggepinnen.github.io/ControlSystemIdentification.jl/stable/ss/', 'System identification, subspace methods']
]
columns = ['Source', 'Used For/Reason']

ack_df = pd.DataFrame(acknowledgements, columns=columns)

ack_df

,Source,Used For/Reason
0,1. https://docs.scipy.org/doc/scipy-1.14.0/ref...,SciPy Documentation for Discrete Time Algebrai...
1,2. https://python-control.readthedocs.io/en/0....,"Control.dare, Riccati Equation, documentation ..."
2,3. https://cruxponent.com/post/lqr/,Tutorials on LQR and Riccati
3,4. https://www.mathworks.com/help/ident/ug/sta...,"System identification, using linear regression..."
4,5. https://baggepinnen.github.io/ControlSystem...,"System identification, subspace methods"


In [20]:

ack_df.to_csv("acknowledgements.txt", sep="\t", index=False)